# Cleaning the time formats in the play-by-play data

Author: Justin Brown

Dated: 08/05/2025

---

The idea for cleaning the time formats is a 3-step process:
1. Identify patterns in the 'WCTIMESTRING' column that can be translated into a standard format. 
2. Translate those patterns into a standard format, such as 12-hour standard.
3. Exclude cleaned games that have weird event densities, such as too many events in a short time period, or too few events in a long time period.



## Finding WCTIMESTRING Patterns

The 'WCTIMESTRING' column do not all have the same format. However, some formats are consistent across the entire game, and may be translated into a standard format that could be used for calculating game durations accurately. 

In order to know if a format is translateable, we need to first identify if there is a way to represent it as a 
regular expression. If we can, and the pattern is hyperspecific, then it's more likely that the format can be translated.

A few differentiating factors between time formats to consider are:
- **Hour Range**: Does the hour range from 0-23, 1-12, or 0-99, or 0-259, or is it unbounded?
- **Minute Range**: Does the minute range from 0-59 or does it not bounded to 60?
- **AM/PM Presence**: Does the time format always include AM/PM, sometimes include it, or never include it?
- **AM/PM Turnover**: Does the time format turnover between AM and PM at 12:00 PM, 12:00 AM, 0:00 AM, or 24:00 AM?
- **AM/PM String Consistency**: Can 'AM' have random instances of 'A', 'PM' have random instances of 'P', or are they always consistent? If they are consistent, can they be interchanged?
- **Negative Hours**: Does the time format allow negative hours, such as -1:00 AM, -2:00 PM, or -99:00 AM?
- **Negative Minutes**: Does the time format allow negative minutes, such as 0:0-30 AM, 0:0-30 PM, or -1:0-30 AM, or 0:-30 PM?
- **Zero-Padding**: Does the time format allow zero-padding, such as 01:00 AM, 01:00 PM, or does it not allow it, such as 1:00 AM, 1:00 PM?
- **Sticky AM/PM**: Does the time format remain PM once it passes a potential turnover point such as 12:00 PM, such as 13:00 PM, 25:00 PM, or 101:00 PM?

### Identifying Patterns

TODO: 
- Break down regex patterns into smaller patterns, and use combinations to match complex cases. 
- Example: 12_hour_standard + changing AM -> A. This would be its own pattern, and if any time doesn't match either format, 
- it would try the next combination. If a combination is found, we convert it to a standard, and then check for time jumps on a case by case basis. 

The cell below is a list of the currently best known patterns patterns that match the time formats found in the data.

They may not be comprehensive or completely accurate and are a work in progress.

In [ ]:
# Regex patterns for time formats found in the data and have a consistent pattern that can be translated into normal durations.
TIME_FORMATS = {
    '12_standard': r'^(?:0?[1-9]|1[0-2]):[0-5][0-9] [AP]M$', # Standard 12-hour
    '24_standard': r'^(?:0?[0-9]|1[0-9]|2[0-3]):[0-5][0-9]$', # Standard 24-hour
    '24_am_pm_with_sticky_pm': r'^(?:0[0-9]|1[0-1]|1[3-9]|2[0-3]):[0-5][0-9] [AP]M$|^12:[0-5][0-9] PM$', # 24-hour always including AM/PM, 12 only allowed with PM
    '24_optional_am_pm': r'^(?:0?[0-9]|1[0-1]|1[3-9]|2[0-3]):[0-5][0-9](?: ?[AP]M)?$|^12:[0-5][0-9]$|^12:[0-5][0-9] ?PM$', # 24-hour sometimes with AM/PM
    '259_hr_both_am_pm': r'^(?:\d{1,3}):[0-5]\d (?:AM|PM)$', # Forms 11:42 AM, 12:00 PM, 1:01 PM, 176:57 AM, can have inconsistent turnover.
    '99_hr_am_pm_with_sticky_pm': r'^(?!12:00 AM$)(?!24:00 AM)[0-9]{2}:[0-5][0-9] ?[AP]M$', # 99-hour always with AM/PM
    '99_hr_optional_am_pm': r'^(?!12:00 AM$)(?:0?[0-9]|[1-9]|[1-9][0-9]|99):[0-5][0-9](?: ?[AP]M)?$', # 99-hour sometimes with AM/PM
    '0_or_1_hr_0_minus_minutes_am_only_with_turnover': r'^(?:0:(?:0-(?:[1-5]?\d)|0{2}|[0-5]?\d)|1:(?:[0-5]\d)) AM$', # Forms 0:0-XX AM or 1:XX AM
    '0_or_1_hr_0_minus_minutes_pm_only_with_turnover': r'^(?:0:(?:0-(?:[1-5]?\d)|0{2}|[0-5]?\d)|1:(?:[0-5]\d)) PM$', # Forms 0:0-XX PM or 1:XX PM
    '0_hr_0_minus_minutes_turnover': r'^0:(?:0(?:-[0-5][0-9])?|[0-5][0-9]) [AP]M$', # Forms 0:0-XX or 0:XX or 0:0-X with both AM and PM
    '259_hr_am_pm_with_sticky_pm': r'^(?:0:[0-5][0-9]|[1-9]:[0-5][0-9]|1[0-1]:[0-5][0-9]) AM$|^(?:12:[0-5][0-9]|1[3-9]:[0-5][0-9]|[2-9][0-9]:[0-5][0-9]|1[0-9]{2}:[0-5][0-9]|2[0-5][0-9]:[0-5][0-9]) PM$', # Forms 0:00 AM, 11:59 AM, 12:00 PM, ... 259:59 PM
    'minus_12_hr_0_minus_minutes_am_or_a_no_turnover': r'^-\d{1,2}:0-(?:[1-5]?\d) A$|^-\d{1,2}:0-(?:[1-5]?\d) AM$|^-\d{1,2}:0{2} A$|^-\d{1,2}:0{2} AM$|^-\d{1,2}:0-(?:[1-9]|[1-5]\d) A$|^-\d{1,2}:0-(?:[1-9]|[1-5]\d) AM$|^-\d{1,2}:0-(?:[1-9]|[1-5]\d) A$|^-\d{1,2}:0-(?:[1-9]|[1-5]\d) AM$|^-\d{1,2}:00 AM$', # Forms -10:0-XX A or -10:0-XX AM, -10:00 A or -10:00 AM
    'minus_199_hr_0_minus_minutes_optional_am_or_a': r"^-1\d{2}:(?:0-(?:[1-5]?\d)|00)(?: ?A(?:M)?)?$", # Matches: -180:0-25, -180:0-9 A, -179:00 AM, etc.
    'minus_12_hr_0_minus_minutes_positive_00_minutes_optional_am_or_a_with_4am_exception_and_minus_00_minutes_am_only_exception': r"^(?:-\d{1,2}:0-(?:[1-5]?\d) ?A(?:M)?|-\d{1,2}:00 AM|4:00 AM)$", # Matches: -10:0-48 A, -10:0-9 AM, -10:00 AM, -9:00 AM, and 4:00 AM NOT -9:00 A.
}

In [101]:
import pandas as pd

def check_time_formats(df: pd.DataFrame, column_name: str) -> str:
    """Find the format of the time strings in the specified column or None if no match found."""
    for format_name, pattern in TIME_FORMATS.items():
        if df[column_name].str.contains(pattern, regex=True).all():
            return format_name
    return "None"
        
def find_non_matching_times(df: pd.DataFrame, column_name: str, time_format: str) -> pd.DataFrame:
    """Returns all rows where the specified column does not match the given time format"""
    pattern = TIME_FORMATS[time_format]
    matches = df[column_name].str.contains(pattern, regex=True)
    non_matching_times = df[~matches]
    return non_matching_times

def find_first_game_with_no_time_format(seasons: list[str], league: str) -> pd.DataFrame:
    """
    Find the first game that doesn't match any mapped pattern in the WCTIMESTRING column.

    Args:
        seasons (list[str]): List of seasons to check. '1996-97', '1997-98', etc.
    Returns:
        pd.DataFrame: DataFrame containing the rows with non-matching time formats.
    """
    for season in seasons:
        season_df = pd.read_csv(f'../pbp_data/{league}/{season}/combined_{season}.csv')
        for game_id, group in season_df.groupby('GAME_ID'):
            time_format = check_time_formats(group, 'WCTIMESTRING')
            if time_format is None:
                print(f"GAME_ID is {game_id}", flush=True)
                print(f"Game ID {game_id} in season {season} has no matching time format")
                print(group['WCTIMESTRING'].unique())
                return find_non_matching_times(group, 'WCTIMESTRING', '12_standard')
            
        print(f"All game_ids have matching time formats in season {season}")
        return pd.DataFrame()


Running the cell below will search all specified seasons for the first game in which no time format is found, 
essentially discovering a new format not previously considered. 

Using that information, another regular expression can be added to the 'TIME_FORMATS' dictionary in an attempt to 
uniquely match that format.

In [103]:
# Edit these variables to test specific datasets
CHOSEN_SEASONS = ['2000-01']
CHOSEN_LEAGUE = 'nba'

# Run the function to find the first game with no matching format. If none is found, it will return an empty DataFrame.
find_first_game_with_no_time_format(CHOSEN_SEASONS, CHOSEN_LEAGUE)

All game_ids have matching time formats in season 2000-01


Empty DataFrame
Columns: []
Index: []

### Plotting the Patterns

In [104]:
import matplotlib.pyplot as plt
import seaborn as sns

def get_time_patterns_count(season_path: str) -> dict:
    """
    Get the frequencies of each time format/pattern for the given season.

    Args:
        season_path (str): Path to the combined season CSV file.
    Returns:
        dict: A dictionary with time format names as keys and their counts as values.
    """
    season_df = pd.read_csv(season_path)
    season_time_patterns = {}
    for game_id, group in season_df.groupby('GAME_ID'):
        time_format = check_time_formats(group, 'WCTIMESTRING')
        season_time_patterns[time_format] = season_time_patterns.get(time_format, 0) + 1
    return season_time_patterns

def plot_time_patterns(season_time_patterns: dict) -> None:
    """
    Plot a single season's time format frequencies as a bar chart.
    Args:
        season_time_patterns (dict): A dictionary with time format names as keys and their counts as values.
    """
    formats = list(season_time_patterns.keys())
    counts = list(season_time_patterns.values())
    plt.figure(figsize=(12, 6))
    sns.barplot(x=formats, y=counts)
    plt.xlabel('Time Format')
    plt.ylabel('Count of Games')
    plt.title('Time Format Frequencies in Season')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def get_total_season_time_patterns(seasons: str) -> dict:
    """
    Get the total frequencies of each time format/pattern across all seasons.

    Args:
        seasons (list): List of season strings.
    Returns:
        dict: A dictionary with time format names as keys and their total counts as values.
    """
    all_season_time_patterns = {}
    for season in seasons:
        print(f"Processing season {season}", flush=True)
        season_path = f'../pbp_data/nba/{season}/combined_{season}.csv'
        season_time_patterns = get_time_patterns_count(season_path)
        for format_name, count in season_time_patterns.items():
            all_season_time_patterns[format_name] = all_season_time_patterns.get(format_name, 0) + count
    
    # Print the total counts for each time format
    for format_name, count in all_season_time_patterns.items():
        print(f"Time Format: {format_name}, Total Count: {count}", flush=True)
    
    return all_season_time_patterns

Running the cell below will plot the distribution of formats found in the specified seasons.

In [105]:
CHOSEN_SEASONS = ['1996-97', '1997-98', '1998-99', '1999-00']
# CHOSEN_SEASONS = os.listdir('../pbp_data/nba/') # Uncomment to process all seasons

plot_time_patterns(get_total_season_time_patterns(CHOSEN_SEASONS))

Processing season 1996-97
Processing season 1997-98
Processing season 1998-99
Processing season 1999-00
Time Format: 24_am_pm_with_sticky_pm, Total Count: 1030
Time Format: 12_standard, Total Count: 3416
Time Format: 259_hr_both_am_pm, Total Count: 104
Time Format: 0_or_1_hr_0_minus_minutes_am_only_with_turnover, Total Count: 1
Time Format: 24_optional_am_pm, Total Count: 23
Time Format: minus_12_hr_0_minus_minutes_am_or_a_no_turnover, Total Count: 1
Time Format: minus_199_hr_0_minus_minutes_optional_am_or_a, Total Count: 1


### Testing the Patterns

The below test suite is intended to verify that the regular expressions for time formats are functioning correctly.

In [106]:
# Example test cases for the regex patterns in order
# TODO: Extensively test pattern combinations rather than repeating the same patterns.
# TODO: Example: Define 12-hour pattern examples, 24-hour pattern examples, 12-hour with AM/PM, etc.
# TODO: Then, test combinations of the defined patterns.

import unittest

class TestTimeFormats(unittest.TestCase):
    def test_12_standard(self):
        valid = ['01:00 AM', '9:30 AM', '11:59 AM', '12:00 PM', '11:45 PM', '12:00 AM']
        invalid = ['13:00 AM', '00:00 PM', '12:60 AM', '12:61 PM', '12:00', '99:00 AM', '99:00', 'AD32:00 PM', '12:00 AM PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['12_standard'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['12_standard'])

    def test_24_standard(self):
        valid = ['00:00', '01:00', '12:30', '23:59', '22:15', '15:00', '09:05', '10:00']
        invalid = ['24:00', '12:60', '12:61', '99:00', '99:59', '12:00 AM', '12:00 PM', '24:00 AM', '99:00', '99:00 PM', '12:00 AM PM', 'AD32:00 PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['24_standard'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['24_standard'])

    def test_24_am_pm_with_sticky_pm(self):
        valid = ['00:00 AM', '01:00 AM', '12:30 PM', '23:59 PM', '22:15 PM', '15:00 PM', '09:05 AM', '10:00 PM']
        invalid = ['24:00 AM', '12:60 PM', '12:61 AM', '99:00 PM', '99:59 AM', '12:00 AM', '24:00 PM', '99:00 AM', '99:00 AM', 'AD32:00 PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['24_am_pm_with_sticky_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['24_am_pm_with_sticky_pm'])

    def test_24_optional_am_pm(self):
        valid = ['00:00', '01:00', '12:30', '23:59', '22:15', '15:00', '09:05', '10:00', '01:00 AM', '12:30 PM']
        invalid = ['24:00', '12:60', '12:61', '99:00', '99:59', '12:00 AM', '24:00 AM', '99:00 PM', '99:00 AM', 'AD32:00 PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['24_optional_am_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['24_optional_am_pm'])
    
    def test_99_hr_am_pm_with_sticky_pm(self):
        valid = ['00:00 AM', '01:00 AM', '12:30 PM', '23:59 PM', '22:15 PM', '15:00 PM', '09:05 AM', '10:00 PM',
                 '99:00 PM', '99:59 PM', '99:30 PM', '99:59 PM']
        invalid = ['24:00 AM', '12:60 PM', '12:61 AM', '100:59 AM', '12:00 AM',
                   '100:00 AM', '100:00 PM', 'AD32:00 PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['99_hr_am_pm_with_sticky_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['99_hr_am_pm_with_sticky_pm'])
    
    def test_99_hr_optional_am_pm(self):
        valid = ['00:00', '01:00', '12:30', '23:59', '22:15', '15:00', '09:05', '10:00',
                 '99:00', '99:59', '99:30', '99:59 PM', '09:05 AM', '10:00 PM']
        invalid = ['12:60', '12:61', '100:59 AM', '12:00 AM',
                   '100:00 AM', '100:00 PM', 'AD32:00 PM']
        
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['99_hr_optional_am_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['99_hr_optional_am_pm'])

    def test_0_or_1_hr_0_minus_minutes_am_only_with_turnover(self):
        valid = ['0:0-51 AM', '0:58 AM', '0:0-45 AM', '0:0-20 AM', '1:58 AM', '0:0-51 AM', '0:0-20 AM']
        invalid = ['0:1-60 AM', '0:0-60 AM', '0:0-61 PM', '0:0-61 AM', '0:0-60 PM',
                   '0:0-30 PM', '0:0-29 PM', '0:0-59 PM', '1:0-51 PM', '1:0-60 AM',
                   '0:0-00 PM', '1:0-51 PM', '1:0-60 PM', '1:0-61 AM', '1:0-61 PM', '1:0-60 AM',]
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['0_or_1_hr_0_minus_minutes_am_only_with_turnover'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['0_or_1_hr_0_minus_minutes_am_only_with_turnover'])

    def test_0_or_1_hr_0_minus_minutes_pm_only_with_turnover(self):
        valid = ['0:0-51 PM', '0:58 PM', '0:0-45 PM', '0:0-20 PM', '1:58 PM', '0:0-51 PM', '0:0-20 PM']
        invalid = ['0:1-60 PM', '0:0-60 PM', '0:0-61 AM', '0:0-61 PM', '0:0-60 AM',
                   '0:0-30 AM', '0:0-29 AM', '0:0-59 AM', '1:0-51 AM', '1:0-60 PM',
                   '0:0-00 AM', '1:0-51 PM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['0_or_1_hr_0_minus_minutes_pm_only_with_turnover'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['0_or_1_hr_0_minus_minutes_pm_only_with_turnover'])

    def test_0_hr_0_minus_minutes_turnover(self):
        valid = ['0:0-51 AM', '0:58 AM', '0:0-45 PM', '0:0-20 PM', '0:0-51 PM', '0:58 PM', '0:0-45 AM', '0:0-20 AM']
        invalid = ['0:1-51 AM', '0:1-51 PM', '0:1-51', '0:1-60 PM', '0:0-60 PM', '0:0-61 AM', '0:0-61 PM', '0:0-60 AM', '0:1-60 AM',
                   '1:0-51 AM', '1:0-51 PM', '1:0-51', '1:0-60 PM', '1:0-61 AM', '1:0-61 PM', '1:0-60 AM',
                   '99:0-51 AM', '99:0-51 PM', '99:0-51', '99:0-60 PM', '99:0-61 AM', '99:0-61 PM', '99:0-60 AM',
                   '12:0-51 AM', '12:0-51 PM', '12:0-51', '12:0-60 PM', '12:0-61 AM', '12:0-61 PM', '12:0-60 AM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['0_hr_0_minus_minutes_turnover'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['0_hr_0_minus_minutes_turnover'])

    def test_259_hr_am_pm_with_sticky_pm(self):
        valid = [
            "4:00 AM", "0:00 AM", "11:59 AM", "12:00 PM", "16:00 PM", "23:59 PM", "24:00 PM", "99:00 PM", "99:59 PM", "100:00 PM",
            "199:59 PM", "200:16 PM", "200:20 PM"
        ]
        invalid = [
            "13:00 AM", "12:00 AM", "0:00 PM", "200:20 AM", "200:16 AM", "200:00 AM", "1:00 A", "1:00", "12:00", "200:00"
        ]
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['259_hr_am_pm_with_sticky_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['259_hr_am_pm_with_sticky_pm'])

    def test_minus_12_hr_0_minus_minutes_am_or_a_no_turnover(self):
        valid = ['-10:0-7 AM', '-10:0-6 AM', '-10:0-5 AM', '-10:0-3 AM', '-10:0-2 AM', '-10:0-1 AM', '-10:00 AM',
            '-9:0-59 AM', '-9:0-57 AM', '-9:0-56 AM', '-9:0-55 AM', '-9:0-54 AM', '-9:0-53 AM', '-9:0-52 AM',
            '-9:0-50 AM', '-9:0-49 AM', '-9:0-48 AM', '-9:0-47 AM', '-9:0-46 AM', '-9:0-45 AM', '-9:0-44 AM',
            '-9:0-43 AM', '-9:0-42 AM', '-9:0-41 AM', '-9:0-40 AM', '-0:0-10 AM', '-0:0-10 A', '-0:0-3 AM', '-0:0-3 A']
        invalid = ['0:00 AM', '11:59 AM', '12:00 PM', '23:59 PM', '25:00 PM', '259:59 PM', '0:0-20 AM',
            '1:05 AM', '-0:05 AM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['minus_12_hr_0_minus_minutes_am_or_a_no_turnover'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['minus_12_hr_0_minus_minutes_am_or_a_no_turnover'])

    def test_259_hr_both_am_pm(self):
        valid = [
            "0:00 AM", "11:58 AM", "11:59 AM", "12:00 PM", "12:01 PM", "1:51 PM", "25:59 PM", "25:30 AM",
            "176:57 AM", "259:59 PM", "259:59 AM"
        ]
        invalid = ['0:00 A', '12:50 P', '12:00', '', '-9:0-40 AM', '-0:0-10 AM', '-0:0-10 A', '-0:0-3 AM', '-0:0-3 A', '0:0-20 AM', '-0:05 AM']
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['259_hr_both_am_pm'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['259_hr_both_am_pm'])

    def test_minus_199_hr_0_minus_minutes_optional_am(self):
        valid = [
            "-180:0-25", "-180:0-9 A", "-179:00 AM", "-199:0-59", "-199:0-58 A", "-199:0-57 AM"
        ]

        invalid = [
            "-180:0-25 PM", "-179:00 PM", "-199:0-59 PM", "-199:0-58 PM", "-199:0-57 PM",
            "-180:0-25 P", "-179:00 P", "-199:0-59 P", "-199:0-58 P", "-199:0-57 P",
            "-200:0-59", "-200:0-58 A", "-200:0-57 AM", "-200:0-56 PM", "-200:0-55 P",
            "4:00 AM", "4:00 PM", "4:00 A", "4:00 P", "4:00", "12:00 AM", "12:00 PM", "12:00 A", "12:00 P",
            "0:00 AM", "0:00 PM", "0:00 A", "0:00 P", "13:00 AM", "13:00 PM", "13:00 A", "13:00 P",
            "24:00 AM", "24:00 PM", "24:00 A", "24:00 P", "25:00 PM", "25:00 P", "25:00",
            "99:00 PM", "99:00 P", "99:00", "100:00 PM", "100:00 P", "100:00"
        ]
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['minus_199_hr_0_minus_minutes_optional_am_or_a'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['minus_199_hr_0_minus_minutes_optional_am_or_a'])

    def test_minus_12_hr_0_minus_minutes_positive_00_minutes_optional_am_or_a_with_4am_exception_and_minus_00_minutes_am_only_exception(self):
        valid = [
            "-10:0-48 A", "-10:0-9 AM", "-9:0-59 AM", "-8:0-44 AM", "4:00 AM", "-10:00 AM", "-9:00 AM"
        ]
        invalid = [
            "-10:0-48 PM", "-10:0-9 PM", "-9:0-59 PM", "-8:0-44 PM", "-10:0-62 AM", "-10:0-62 PM",
            "-10:0-62 A", "-10:0-62 P", "-10:0-61 AM", "-10:0-61 PM", "-10:0-61 A", "-10:0-61 P",
            "4:00 PM", "-10:0-48 P", "-10:0-9 P", "-9:0-59 P", "-8:0-44 P", "4:00 P", "11:59 AM",
            "11:59 A", "11:59", "12:01 PM", "12:01 P", "12:01", "12:00 AM", "12:00 A", "12:00", 
            "0:00 AM", "0:00 A", "0:00", "13:00 AM", "13:00 A", "13:00", "13:01 PM", "13:01 P", "13:01",
            "24:00 AM", "24:00 A", "24:00", "24:01 PM", "24:01 P", "24:01",
            "25:00 PM", "25:00 P", "25:00", "99:00 PM", "99:00 P", "99:00", "100:00 PM", "100:00 P", "100:00"
        ]
        for time in valid:
            self.assertRegex(time, TIME_FORMATS['minus_12_hr_0_minus_minutes_positive_00_minutes_optional_am_or_a_with_4am_exception_and_minus_00_minutes_am_only_exception'])
        for time in invalid:
            self.assertNotRegex(time, TIME_FORMATS['minus_12_hr_0_minus_minutes_positive_00_minutes_optional_am_or_a_with_4am_exception_and_minus_00_minutes_am_only_exception'])

    unittest.main(argv=[''], verbosity=2, exit=False)

test_0_hr_0_minus_minutes_turnover (__main__.TestTimeFormats.test_0_hr_0_minus_minutes_turnover) ... ok
test_0_or_1_hr_0_minus_minutes_am_only_with_turnover (__main__.TestTimeFormats.test_0_or_1_hr_0_minus_minutes_am_only_with_turnover) ... ok
test_0_or_1_hr_0_minus_minutes_pm_only_with_turnover (__main__.TestTimeFormats.test_0_or_1_hr_0_minus_minutes_pm_only_with_turnover) ... ok
test_12_standard (__main__.TestTimeFormats.test_12_standard) ... ok
test_24_am_pm_with_sticky_pm (__main__.TestTimeFormats.test_24_am_pm_with_sticky_pm) ... ok
test_24_optional_am_pm (__main__.TestTimeFormats.test_24_optional_am_pm) ... ok
test_24_standard (__main__.TestTimeFormats.test_24_standard) ... ok
test_259_hr_am_pm_with_sticky_pm (__main__.TestTimeFormats.test_259_hr_am_pm_with_sticky_pm) ... ok
test_259_hr_both_am_pm (__main__.TestTimeFormats.test_259_hr_both_am_pm) ... ok
test_99_hr_am_pm_with_sticky_pm (__main__.TestTimeFormats.test_99_hr_am_pm_with_sticky_pm) ... ok
test_99_hr_optional_am_pm (__m